In [1]:
import pandas as pd

# Process input

In [32]:
df_meta = pd.read_csv('meta_data.csv', index_col=0)
df_meta = df_meta[(df_meta['characteristics_ch1.2.genotype']=='C57BL6/J') &
                  (df_meta['characteristics_ch1.4.infection']=='Mock infected')]
df_meta = df_meta[['title','characteristics_ch1.3.treatment','characteristics_ch1.5.hour post infection','characteristics_ch1.6.mouse identifier','characteristics_ch1.7.sequencing']]
# use BGI since Psomagen only inlcudes untreated mice
df_meta = df_meta[df_meta['characteristics_ch1.7.sequencing']=='BGI']
df_meta

,title,characteristics_ch1.3.treatment,characteristics_ch1.5.hour post infection,characteristics_ch1.6.mouse identifier,characteristics_ch1.7.sequencing
GSM4955498,gso32.BL6.N.N.4h.1,Untreated,4,1,BGI
GSM4955499,gso32.BL6.IFNb.N.4h.1,IFNbeta,4,1,BGI
GSM4955504,gso32.BL6.N.N.4h.2,Untreated,4,2,BGI
GSM4955505,gso32.BL6.IFNb.N.4h.2,IFNbeta,4,2,BGI
GSM4955510,gso32.BL6.N.N.24h.1,Untreated,24,1,BGI
GSM4955511,gso32.BL6.IFNb.N.24h.1,IFNbeta,24,1,BGI
GSM4955516,gso32.BL6.N.N.24h.2,Untreated,24,2,BGI
GSM4955517,gso32.BL6.IFNb.N.24h.2,IFNbeta,24,2,BGI


In [33]:
df_raw_count = pd.read_csv('GSE162620_RNASeq_GO0102_raw_counts_GEO.tsv', sep='\t')
df_raw_count = df_raw_count.drop('gene', axis=1).set_index('Geneid')
df_raw_count = df_raw_count[[c for c in df_raw_count.columns if c in list(df_meta.title)]]
df_raw_count.index.name = 'ensgene'
df_raw_count.to_csv('deseq2_input_raw_count.csv')
df_raw_count

,gso32.BL6.N.N.4h.1,gso32.BL6.IFNb.N.4h.1,gso32.BL6.N.N.4h.2,gso32.BL6.IFNb.N.4h.2,gso32.BL6.N.N.24h.1,gso32.BL6.IFNb.N.24h.1,gso32.BL6.N.N.24h.2,gso32.BL6.IFNb.N.24h.2
ensgene,,,,,,,,
ENSMUSG00000033845,955,460,954,406,629,591,608,553
ENSMUSG00000025903,1163,849,1302,951,1098,1081,1073,1031
ENSMUSG00000033813,786,1291,894,1346,541,717,579,825
ENSMUSG00000033793,2541,2148,2598,1968,1606,1615,1549,1620
ENSMUSG00000025907,992,562,1096,603,1901,1288,1789,1374
...,...,...,...,...,...,...,...,...
ENSMUSG00000065947,228,140,239,155,239,28,235,23
ENSMUSG00000064363,42384,25543,44336,26501,45153,5258,46186,5719
ENSMUSG00000064367,76527,50593,83329,50720,88018,12165,94882,13555


In [34]:
df_meta = df_meta[['title','characteristics_ch1.3.treatment','characteristics_ch1.5.hour post infection','characteristics_ch1.6.mouse identifier']]
df_meta.columns = ['id','treatment','time','mouse']
group = []
for x,y in zip(df_meta['treatment'], df_meta['time']):
    group.append(x+'_'+str(y))
df_meta['group'] = group
df_meta.to_csv('deseq2_input_metadata.csv', index=False)
df_meta

,id,treatment,time,mouse,group
GSM4955498,gso32.BL6.N.N.4h.1,Untreated,4,1,Untreated_4
GSM4955499,gso32.BL6.IFNb.N.4h.1,IFNbeta,4,1,IFNbeta_4
GSM4955504,gso32.BL6.N.N.4h.2,Untreated,4,2,Untreated_4
GSM4955505,gso32.BL6.IFNb.N.4h.2,IFNbeta,4,2,IFNbeta_4
GSM4955510,gso32.BL6.N.N.24h.1,Untreated,24,1,Untreated_24
GSM4955511,gso32.BL6.IFNb.N.24h.1,IFNbeta,24,1,IFNbeta_24
GSM4955516,gso32.BL6.N.N.24h.2,Untreated,24,2,Untreated_24
GSM4955517,gso32.BL6.IFNb.N.24h.2,IFNbeta,24,2,IFNbeta_24


# Process output

In [35]:
df_raw_count = pd.read_csv('GSE162620_RNASeq_GO0102_raw_counts_GEO.tsv', sep='\t')
df_raw_count

,gene,Geneid,gso32.STING.N.N.4h.3,gso32.STING.IFNb.N.4h.3,gso32.STING.N.Rv.4h.3,gso32.STING.IFNb.Rv.4h.3,gso32.STING.N.HK.4h.3,gso32.STING.IFNb.HK.4h.3,gso32.STING.N.N.4h.4,gso32.STING.IFNb.N.4h.4,...,gso32.BL6.N.Rv.24h.1,gso32.BL6.IFNb.Rv.24h.1,gso32.BL6.N.HK.24h.1,gso32.BL6.IFNb.HK.24h.1,gso32.BL6.N.N.24h.2,gso32.BL6.IFNb.N.24h.2,gso32.BL6.N.Rv.24h.2,gso32.BL6.IFNb.Rv.24h.2,gso32.BL6.N.HK.24h.2,gso32.BL6.IFNb.HK.24h.2
0,Mrpl15,ENSMUSG00000033845,1360,485,909,296,1071,387,978,343,...,466,552,742,686,608,553,463,466,727,611
1,Lypla1,ENSMUSG00000025903,404,299,260,162,277,224,251,169,...,1049,940,1252,1177,1073,1031,980,936,1139,1174
2,Tcea1,ENSMUSG00000033813,677,1173,435,706,606,836,459,686,...,1119,1151,643,668,579,825,1137,1159,733,760
3,Atp6v1h,ENSMUSG00000033793,3270,2565,2694,1591,2815,1886,2165,1613,...,2529,2321,1940,1983,1549,1620,2554,2404,1958,2073
4,Rb1cc1,ENSMUSG00000025907,793,529,462,315,513,413,505,303,...,1014,843,1338,1222,1789,1374,913,976,1243,1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12527,mt-Nd4l,ENSMUSG00000065947,393,235,237,120,222,149,225,124,...,48,33,221,65,235,23,47,26,232,43
12528,mt-Nd4,ENSMUSG00000064363,26024,15199,15666,7757,17707,10056,16393,8117,...,11018,4689,40019,10171,46186,5719,10780,5264,40138,9830
12529,mt-Nd5,ENSMUSG00000064367,36809,22187,22198,11365,24498,14908,22931,12206,...,28380,11082,79616,23450,94882,13555,27561,12701,76950,25084
12530,mt-Nd6,ENSMUSG00000064368,12101,7746,6915,3832,8145,5026,7336,3841,...,9625,3873,28617,8253,33869,4717,9347,4015,25679,9390


In [44]:
df_output = pd.read_csv('deseq2_output_wald_t4.csv').rename({'Unnamed: 0':'Geneid'}, axis=1)
df_output = pd.merge(df_output, df_raw_count[['gene','Geneid']], left_on=['Geneid'], right_on=['Geneid'], how='left')
df_output.to_csv('deseq2_output_wald_t4_w_gene_name.csv', index=False)
df_output

,Geneid,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene
0,ENSMUSG00000033845,620.947572,-0.359793,0.112283,-3.204358,1.353640e-03,2.327882e-03,Mrpl15
1,ENSMUSG00000025903,1062.510580,0.329184,0.097382,3.380322,7.240103e-04,1.278452e-03,Lypla1
2,ENSMUSG00000033813,941.843143,1.435172,0.093350,15.374133,2.440896e-53,2.100848e-52,Tcea1
3,ENSMUSG00000033793,1980.754310,0.461292,0.073475,6.278219,3.424722e-10,9.545471e-10,Atp6v1h
4,ENSMUSG00000025907,1149.218871,-0.058806,0.096101,-0.611921,5.405902e-01,5.997353e-01,Rb1cc1
...,...,...,...,...,...,...,...,...
12527,ENSMUSG00000065947,155.974693,0.119660,0.203200,0.588880,5.559419e-01,6.142633e-01,mt-Nd4l
12528,ENSMUSG00000064363,29041.162114,0.045867,0.047680,0.961985,3.360573e-01,3.959772e-01,mt-Nd4
12529,ENSMUSG00000064367,56739.645713,0.126306,0.051251,2.464442,1.372269e-02,2.075224e-02,mt-Nd5
12530,ENSMUSG00000064368,20138.893982,0.205757,0.066084,3.113587,1.848281e-03,3.129532e-03,mt-Nd6


In [45]:
df_output = pd.read_csv('deseq2_output_wald_t24.csv').rename({'Unnamed: 0':'Geneid'}, axis=1)
df_output = pd.merge(df_output, df_raw_count[['gene','Geneid']], left_on=['Geneid'], right_on=['Geneid'], how='left')
df_output.to_csv('deseq2_output_wald_t24_w_gene_name.csv', index=False)
df_output

,Geneid,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene
0,ENSMUSG00000033845,620.947572,0.105980,0.112313,0.943620,3.453641e-01,4.160249e-01,Mrpl15
1,ENSMUSG00000025903,1062.510580,0.180210,0.097080,1.856298,6.341112e-02,9.313833e-02,Lypla1
2,ENSMUSG00000033813,941.843143,0.678167,0.099478,6.817245,9.280313e-12,3.714201e-11,Tcea1
3,ENSMUSG00000033793,1980.754310,0.255420,0.076098,3.356454,7.894893e-04,1.596701e-03,Atp6v1h
4,ENSMUSG00000025907,1149.218871,-0.253255,0.088300,-2.868139,4.128946e-03,7.546234e-03,Rb1cc1
...,...,...,...,...,...,...,...,...
12527,ENSMUSG00000065947,155.974693,-2.996699,0.273785,-10.945435,6.988129e-28,5.339374e-27,mt-Nd4l
12528,ENSMUSG00000064363,29041.162114,-2.838764,0.049213,-57.682699,0.000000e+00,0.000000e+00,mt-Nd4
12529,ENSMUSG00000064367,56739.645713,-2.612142,0.051823,-50.405254,0.000000e+00,0.000000e+00,mt-Nd5
12530,ENSMUSG00000064368,20138.893982,-2.572298,0.067342,-38.197439,0.000000e+00,0.000000e+00,mt-Nd6
